# Configuration & Frames

more detailed docs: https://marctoussaint.github.io/robotics-course/

In [1]:
from robotic import ry

## Setting up a basic Config

The starting point is to create a `Configuration`.

In [2]:
C = ry.Config()
C.view()

0

This shows an empty configuration. Tip: Make the view window appear "Always On Top" (right click on the window bar)

You can add things (objects, scene models, robots) to a configuration.

In [3]:
C.clear()
C.addFile('../rai-robotModels/pr2/pr2.g')
C.addFile('../rai-robotModels/objects/kitchen.g')
C.view()

86

You need to call C.view() to update the view

In [4]:
ball = C.addFrame(name="ball")
ball.setShape(ry.ST.sphere, [.1])
ball.setPosition([.8,.8,1.5])
ball.setColor([1,1,0])
C.view()

86

One can also add convex meshes (just passing the vertex array), or use sphere-swept convex meshes (ssBox, capsule, sphere, etc)

In [5]:
C.addFrame(name="hand", parent="pr2L") \
    .setShape(ry.ST.ssBox, size=[.2,.2,.1,.02]) \
    .setRelativePosition([0,0,-.1]) \
    .setColor([1,1,0])
C.view()

86

In this last example, the new object has another frame (pr2L) as *parent*. This means that it is permanently attached to this parent. pos and quat/rot are interpreted relative to the parent.

In [6]:
f = C.frame("hand")
print("position:", f.getPosition())
print("orientation:", f.getQuaternion())

position: [-0.18142104  0.54312202  1.00970044]
orientation: [ 0.33464431 -0.27540941 -0.72687736  0.53274023]


TODO (below): getters and setters for frames

In [7]:
frameC = C.frame('C')
print('pos:', frameC.getPosition(), 'quat:', frameC.getQuaternion())

-- WARNING:kin.cpp:getFrame:384(-1) cannot find frame named 'C'


AttributeError: 'NoneType' object has no attribute 'getPosition'

In [8]:
q[0] = q[0] + .5
C.setJointState(q)
print('pos:', frameC.getPosition(), 'quat:', frameC.getQuaternion())

NameError: name 'q' is not defined

In [9]:
[y,J] = C.eval(ry.FS.position, ['C'])
print('position of C:', y, '\nJacobian:', J)
type(J)

-- WARNING:kin.cpp:getFrame:384(-1) cannot find frame named 'C'
STACK9 /usr/bin/python3() [0x50b8d3]
STACK8 /usr/bin/python3(_PyObject_MakeTpCall
STACK7 /usr/bin/python3(PyCFunction_Call
STACK6 /home/mtoussai/.local/lib/python3.8/site-packages/robotic/ry.so(+0x7b8cde) [0x7f93c1d7acde]
STACK5 /home/mtoussai/.local/lib/python3.8/site-packages/robotic/ry.so(+0x82f07c) [0x7f93c1df107c]
STACK4 rai::Configuration::eval(FeatureSymbol, rai::Array<rai::String> const&, rai::ArrayDouble const&, rai::ArrayDouble const&, int)
STACK3 rai::Configuration::feature(FeatureSymbol, rai::Array<rai::String> const&, rai::ArrayDouble const&, rai::ArrayDouble const&, int) const
STACK2 symbols2feature(FeatureSymbol, rai::Array<rai::String> const&, rai::Configuration const&, rai::ArrayDouble const&, rai::ArrayDouble const&, int)
STACK1 rai::Configuration::getFrameIDs(rai::Array<rai::String> const&) const
STACK0 rai::LogToken::~LogToken()


== ERROR:kin.cpp:getFrameIDs:415(-2) frame name 'C' doesn't exist


RuntimeError: kin.cpp:getFrameIDs:415(-2) frame name 'C' doesn't exist

In [ ]:
#only the z-position relative to target 0.5:
C.eval(ry.FS.position, ['C'], [[0,0,1]], [0,0,0.5]) #the scaling is a 1x3 matrix

## Joint and Frame State

A configuration is a tree of n frames. Every frame has a pose (position & quaternion), which is represented as a 7D vector (x,y,z, qw,qx,qy,qz). The frame state is the $n\times 7$ matrix, where the i-th row is the pose of the i-th frame.

A configuration also defines joints, which means that the relative transfromation from a parent to a child frame is parameterized by degrees-of-freedoms (DOFs). If the configuration has in total n DOFs, the joint state is a n-dimensional vector.

Setting the joint state implies computing all relative transformations, and then forward chaining all transformations to compute all frame poses. So setting the joint state also sets the frame state.
     
Setting the frame state allows you to set frame poses that are inconsistent/impossible w.r.t. the joints! Setting the frame state implies computing all relative transformations from the frame poses, and then assigning the joint state to the *projection* onto the actual DOFs

In [ ]:
q = C.getJointState()
print('joint names: ', C.getJointNames() )
print('joint state: ', q)

Let's move the configuration by adding to the joint configuration

In [ ]:
q[2] = q[2] + 1.
C.setJointState(q)
C.view()

The *frame state* is a $n\times 7$ matrix, which contains for all of $n$ frames the 7D pose. A pose is stored as [p_x, p_y, p_z, q_w, q_x, q_y, q_z], with position p and quaternion q.

In [ ]:
X0 = C.getFrameState()
print('frame state: ', X0)

Let's do a questionable thing: adding .1 to all numbers in the frame matrix!

In [ ]:
X = X0 + .1
C.setFrameState(X)
C.view()

The rows of X have non-normalized quaternions! These are normalized when setting the frame state.

Also, the frame poses are now *inconsistent* to the joint constraints! We can read out the projected joint state, set the joint state, and get a consistent state again:

In [ ]:
C.setJointState( C.getJointState() )
C.view()

Now all *joint* transformations are consistent: just hingeX transformations or alike. However, all the other relative transformations of links and shapes are still messed up from setting their frame pose. Let's bring the configuration back into the state before the harsh *setFrame*

In [ ]:
C.setFrameState(X0)
C.view()

 ## Selecting joints

Often one would like to choose which joints are actually active, that is, which joints are referred to in q. This allows one to sub-select joints and work only with projections of the full configuration state. This changes the joint state dimensionality, including ordering of entries in q.

The frame state is not affected by such a selection of active joints.

In [ ]:
C.selectJointsByTag(["armL","base"])
q = C.getJointState()
print('joint state: ', q)
print('joint names: ', C.getJointNames() )

## Features & Jacobians

A core part of rai defines features over configurations. A feature is a differentiable mapping from a configuration (or set of configurations) to a vector. Starndard features are "position-of-endeffector-X" or "distance/penetration-between-convex-shapes-A-and-B", etc. But there are many, many more features defined in rai, like error of Newton-Euler-equations for an object, total energy of the system, etc. Defining differentiable features is the core of many functionalities in the rai code.

Let's define a basic feature over C: the 3D (world coordinate) position of pr2L (left hand)

In [ ]:
F = C.feature(ry.FS.position, ["pr2L"])

We can now evaluate the feature, and also get the Jacobian:

In [ ]:
print(F.description(C))

[y,J] = F.eval(C)
print('hand position:', y)
print('Jacobian:', J)
print('Jacobian shape:', J.shape)

We can linearly transform features by setting 'scale' and 'target':

In [ ]:
F2 = C.feature(ry.FS.distance, ["hand", "ball"])
print(F2.description(C))

In [ ]:
F2.eval(C)

## Camera views (needs more testing)

We can also add a frame, attached to the head, which has no shape associated to it, but create a view is associated with that frame:

In [ ]:
C.addFrame(name='camera', parent='head_tilt_link', args='Q:<d(-90 1 0 0) d(180 0 0 1)> focalLength:.3')
V = C.cameraView()
IV = V.imageViewer()
V.addSensor(name='camera', frameAttached='camera', width=600, height=400)

In [ ]:
[image,depth] = V.computeImageAndDepth()

When we move the robot, that view moves with it:

In [ ]:
C.setJointState(q=[0.5], joints=['head_pan_joint'])
V.updateConfig(C)
V.computeImageAndDepth()

To close a view (or destroy a handle to a computational module), we reassign it to zero. We can also remove a frame from the configuration.

In [ ]:
IV = 0
V = 0
C.delFrame('camera')

In [ ]:
C=0